* Reference: https://docs.opensearch.org/docs/latest/clients/python-low-level/

In [17]:
!pip install opensearch-py

Defaulting to user installation because normal site-packages is not writeable


In [26]:
# Connecting to  open Search
from opensearchpy import OpenSearch
import os
from dotenv import load_dotenv
load_dotenv()

# Run the docker
host = 'localhost'
port = 9200
auth = ('admin', os.environ.get('OPENSEARCH_INITIAL_ADMIN_PASSWORD', 'admin')) # For testing only. Don't store credentials in code.
use_ssl=True
verify_certs=False

client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_compress=True,
    http_auth=auth,
    use_ssl=use_ssl,
    verify_certs=verify_certs,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
    timeout=60  # Increase timeout to handle slower connections
)

In [27]:
# creating an index
index_name = 'python-test-index'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 2
    }
  }
}

response = client.indices.create(index_name, body=index_body)

In [28]:
# Indexing the document
document = {
  'title': 'Moneyball',
  'director': 'Bennett Miller',
  'year': '2011'
}

response = client.index(
    index = 'python-test-index',
    body = document,
    id = '1',
    refresh = True
)

## performing bulk operations
* operations may be of same or different type
- oeprations must be single string and seperated by \n

In [31]:

movies = '{ "index" : { "_index" : "my-dsl-index", "_id" : "2" } } \n { "title" : "Interstellar", "director" : "Christopher Nolan", "year" : "2014"} \n { "create" : { "_index" : "my-dsl-index", "_id" : "3" } } \n { "title" : "Star Trek Beyond", "director" : "Justin Lin", "year" : "2015"} \n { "update" : {"_id" : "3", "_index" : "my-dsl-index" } } \n { "doc" : {"year" : "2016"} }'

client.bulk(movies)

{'took': 9,
 'errors': True,
 'items': [{'index': {'_index': 'my-dsl-index',
    '_id': '2',
    '_version': 2,
    'result': 'updated',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 3,
    '_primary_term': 1,
    'status': 200}},
  {'create': {'_index': 'my-dsl-index',
    '_id': '3',
    'status': 409,
    'error': {'type': 'version_conflict_engine_exception',
     'reason': '[3]: version conflict, document already exists (current version [2])',
     'index': 'my-dsl-index',
     'shard': '0',
     'index_uuid': '68VS1SqqTh6t_nOzxwbkow'}}},
  {'update': {'_index': 'my-dsl-index',
    '_id': '3',
    '_version': 2,
    'result': 'noop',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 2,
    '_primary_term': 1,
    'status': 200}}]}

## Searching for documents
* using multi-match query to search for “miller” in the title and director fields
* boost the document that have "miller" in title field

In [32]:

q = 'miller'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'director']
    }
  }
}

response = client.search(
    body = query,
    index = 'python-test-index'
)
response

{'took': 11,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.13076457,
  'hits': [{'_index': 'python-test-index',
    '_id': '1',
    '_score': 0.13076457,
    '_source': {'title': 'Moneyball',
     'director': 'Bennett Miller',
     'year': '2011'}}]}}

## Deleting a document
- we cna delete a document using client.delete

In [33]:
response = client.delete(
    index='python-test-index',
    id = '1'
)

## Deleting an index
- we can delete an index using client.indices.delete

In [34]:
response = client.indices.delete(
    index='python-test-index'
)